<center>

# Implementation of New Message Encryption using Elliptic Curve Cryptography Over Finite Fields
</center>

Used article: [link](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9493519)

## Proposed method parameters

<center>

| Set-up Parameters | *n, a, b, p, G* |
|-------------------|-----------------|
| Key Generator     | *P_B, P_A, n_A, n_B, K* |
| Encryption | *E* = [*c_1c_2c_3...c_l*] |
| Decryption | *M* = [m_1m_2m_3...m_l] |
</center>

## 01. Set-up Parameters

A and B side should agree upon an elliptic curve *E_p(a,b)* over finite fields where *p* is a prime number, *a* and *b* are integers less than *p*.

Points are calculated according to the equation:
$$ y^2 = x^3 + ax + b \mod p$$

Which then needs to satisfy the condition:
$$ 4a^3+27b^2\ne 0 \mod p$$

In [12]:
# count points on the finite field E_1286081
def sqrt(val: int):
    return val**(1/2)

def count_y(x, a, b):
    aux = x**3 + a*x + b
    return aux

def point(x, y, a, b, p):
    result = (x**3 + a*x + b - y**2) % p
    if result == 0:
        # print("Point P({},{}) belongs to curve".format(x, y))
        return True
    else:
        # print("Result:", result)
        return False
    
def map_points(a, b, p):
    for y in reversed(range(p)):
        print("{:2d}".format(y), end=" ")
        for x in range(p):
            result = point(x, y, a, b, p)
            if result == True:
                print("+", end=" ")
            else:
                print(".", end=" ")
        print("")
        
    print("   ", end="")
    for x_label in range(p):
        if x_label >= 10:
            print("{:2d}".format(x_label), end="")
        else:
            print("{}".format(x_label), end=" ")

# print(point(5, 8, 0, 7, 17))

def list_points(a, b, p):
    points = []
    
    for x in range(p):
        for y in range(p):
            result = point(x, y, a, b, p)
            if result == True:
                points.append((x, y))
    return points

map_points(4, 9, 13)

12 . + . . . . . . . . . . . 
11 . . . . . . . . . . . . + 
10 + . . + . . . . . . + . . 
 9 . . . . . . . + . . . . . 
 8 . . + . . . . . . . . . . 
 7 . . . . . . . . . . . . . 
 6 . . . . . . . . . . . . . 
 5 . . + . . . . . . . . . . 
 4 . . . . . . . + . . . . . 
 3 + . . + . . . . . . + . . 
 2 . . . . . . . . . . . . + 
 1 . + . . . . . . . . . . . 
 0 . . . . . . . . . . . . . 
   0 1 2 3 4 5 6 7 8 9 101112

In [13]:
from tinyec.ec import SubGroup, Curve

a = 7
b = 3
p = 13

points = list_points(a, b, p)
print(points)


field = SubGroup(p=13, g=(3, 5), n=13, h=1)
curve = Curve(a=7, b=3, field=field, name='p1373')

# for curr_point in points:
#     field = SubGroup(p=p, g=curr_point, n=len(points)+1, h=1)
#     curve = Curve(a=a, b=b, field=field, name='ppp')
#     counter = 1
#     for k in range(1, 20):
#         p = k * curve.g
#         if p.x != None and p.y != None:
#             counter += 1
#         else:
#             break
#     print("{} | {}".format(curr_point, counter))


counter = 1
for k in range(10, 30):
    p = k * curve.g
    print(f"{k} * G = ({p.x}, {p.y})")
    if p.x != None and p.y != None:
        counter += 1
    # else:
    #     break
print(counter)

[(0, 4), (0, 9), (2, 5), (2, 8), (3, 5), (3, 8), (4, 2), (4, 11), (6, 1), (6, 12), (8, 5), (8, 8)]
10 * G = (2, 8)
11 * G = (4, 11)
12 * G = (3, 8)
13 * G = (None, None)
14 * G = (3, 5)
15 * G = (4, 2)
16 * G = (2, 5)
17 * G = (8, 8)
18 * G = (6, 1)
19 * G = (0, 4)
20 * G = (0, 9)
21 * G = (6, 12)
22 * G = (8, 5)
23 * G = (2, 8)
24 * G = (4, 11)
25 * G = (3, 8)
26 * G = (None, None)
27 * G = (3, 5)
28 * G = (4, 2)
29 * G = (2, 5)
19


## Adding points on a curve

### Points addition

> Let define two points on the elliptic curve *E*(*a*,*b*).
> These points are two distinct points $P(x_1,y_1)$ and $Q(x_2,y_2)$.\
> The calculation of the sum of these two points is given as follows.

$$ R(x_3,y_3)=P(x_1,y_1)+Q(x_2,y_2) $$

* if $x_1 \ne x_2$, the sum in this case is defined by

$$ x_3 = \lambda^2-x_1-x_2 $$
$$ y_3 = \lambda(x_1-x_3)-y_1 $$

where

$$ \lambda = \frac{y_2-y_1}{x_2-x_1} $$

* if $x_1 = x_2$, this case gives the point at infinity ($\theta$)

### Points doubling

> Let define two points on the elliptic curve *E*(*a*,*b*).
> These points are $P(x_1,y_1)$ and $Q(x_2,y_2)$, which are equal to each other i.e. $P=Q$.\
> Point doubling can be defined as follows.

$$ P(x_1,y_1)=Q(x_2,y_2) $$

where

$$ R(x_3,y_3)=P(x_1,y_1)+P(x_1,y_1)=2P $$

The calculation of the point doubling is given as follows.

$$ x_3 = \lambda^2-2x_1 $$
$$ y_3 = \lambda(x_1-x_3)-y_1 $$

where:

$$ \lambda = \frac{3x_1^2+a}{2y_1} \mod p $$



In [14]:
def calculate_modular_inverse(base, modulus):
    # Calculate the modular inverse using Extended Euclidean Algorithm
    a = base
    b = modulus
    x, y = 1, 0
    while b != 0:
        q = a // b
        a, b = b, a % b
        x, y = y, x - q * y
    
    if a == 1:
        # Ensure the modular inverse exists
        return x % modulus
    else:
        # Modular inverse does not exist
        return None

def sum_pt(a, p, point_1, point_2 = (None, None)):
    
    # if there occurs point doubling
    if point_2[0] == None or point_2[1] == None:
        lam = ((3*point_1[0]**2 + a) * calculate_modular_inverse(2*point_1[1], p)) % p
        # print("LAMBDA:", lam)
        
        x_3 = (lam**2 - 2*point_1[0]) % p
        y_3 = (lam*(point_1[0] - x_3) - point_1[1]) % p
        
        return (x_3, y_3)
    
    if point_1[0] == point_2[0]:
        # returns infinity
        return (None, None)
    
    lam = ((point_2[1] - point_1[1]) * calculate_modular_inverse(point_2[0]-point_1[0], p)) % p
    
    x_3 = (lam**2 - point_1[0] - point_2[0]) % p
    y_3 = (lam*(point_1[0] - x_3) - point_1[1]) % p
    
    return (x_3, y_3)

def mul_pt(k, a, p, G):
    # print(k)
    # print("P({}, {})".format(G[0], G[1]))
    
    if k == 0:
        return (None, None)
    elif k == 1:
        return G
    elif k % 2 == 1:
        return sum_pt(a, p, G, mul_pt(k-1, a, p, G))
    else:
        return mul_pt(k//2, a, p, sum_pt(a, p, G))

init_point = (3, 5)

for it in range(30):
    print(it, mul_pt(it, 7, 13, init_point))
#-----------------

# Y^2 = X^3 + 497x + 1768; p = 9739

# p_1 = (5274, 2841)
# p_2 = (8669, 740)
# print("Is it working? ", point(5274, 2841, 497, 1768, 9739))
# print("2X:", sum_pt(497, 9739, (5274, 2841)))
# print("---")


print("--- Montgomery Ladder algorithm ---")
# print("96G:", mul_pt(96, 7, 13, init_point))
# print("1G:", mul_pt(1, 7, 13, init_point))
# print("2G:", mul_pt(2, 7, 13, init_point))
# print("3G:", mul_pt(3, 7, 13, init_point))
# print("4G:", mul_pt(4, 7, 13, init_point))




# 1 * G = (0, 9)
# 2 * G = (3, 5)
# 3 * G = (6, 12)
# 4 * G = (4, 2)
# 5 * G = (8, 5)
# 6 * G = (2, 5)
# 7 * G = (2, 8)
# 8 * G = (8, 8)
# 9 * G = (4, 11)
# 10 * G = (6, 1)
# 11 * G = (3, 8)
# 12 * G = (0, 4)
# 13 * G = (None, None)

0 (None, None)
1 (3, 5)
2 (4, 2)
3 (2, 5)
4 (8, 8)
5 (6, 1)
6 (0, 4)
7 (0, 9)
8 (6, 12)
9 (8, 5)
10 (2, 8)
11 (4, 11)
12 (3, 8)
13 (None, None)
14 (3, 5)
15 (None, None)
16 (2, 5)
17 (8, 8)
18 (6, 1)
19 (0, 4)
20 (0, 9)
21 (6, 12)
22 (8, 5)
23 (2, 8)
24 (4, 11)
25 (3, 8)
26 (None, None)
27 (4, 2)
28 (4, 2)
29 (2, 5)
--- Montgomery Ladder algorithm ---


In [15]:
from tinyec import registry

class Point:
    def __init__(self, x, y, curve = None) -> None:
        self.curve = curve
        self.x = x
        self.y = y
        if curve == None:
            self.on_curve = None
        elif x == None or y == None:
            self.on_curve = None
        else:
            self.on_curve = curve.on_curve(x, y)
        pass
    
    def __add_pt(self, point_1, point_2):
        # INFO: print for DEBUG purposes ONLY!
        # print("ADD pts {} + {}".format(point_1, point_2))
        if point_1 == point_2:
            return self.__double_pt(point_1)
        if point_1 == None:
            return point_2
        elif point_2 == None:
            return point_1
        elif point_1.x == point_2.x and point_1.y != point_2.y:
            return Point(None, None, self.curve)
        
        lambda_f = ((point_2.y - point_1.y) * pow(point_2.x - point_1.x, -1, self.curve.p)) % self.curve.p
        
        x_3 = (lambda_f**2 - point_1.x - point_2.x) % self.curve.p
        y_3 = (lambda_f * (point_1.x - x_3) - point_1.y) % self.curve.p
        
        return Point(x_3, y_3, self.curve)

    def __double_pt(self, point):
        # INFO: print for DEBUG purposes ONLY!
        # print("DOUBLE pt {}".format(point))
        if point == None:
            return Point(None, None, self.curve)
        elif point.y == 0:
            return Point(None, None, self.curve)
        
        lambda_f = ((3*pow(point.x, 2) + self.curve.a) * pow(2 * point.y, -1, self.curve.p)) % self.curve.p
        
        x_3 = (lambda_f**2 - 2*point.x) % self.curve.p
        y_3 = (lambda_f * (point.x - x_3) - point.y) % self.curve.p
        
        return Point(x_3, y_3, self.curve)
    
    def __mul_pt(self, point, multiplier):
        if multiplier == 0:
            return Point(None, None, self.curve)
        elif multiplier == 1:
            return point
        elif multiplier % 2 == 1:
            return self.__add_pt(point, self.__mul_pt(point, multiplier-1))
        else:
            return self.__mul_pt(self.__double_pt(point), multiplier//2)

    def __mul__(self, multiplier):        
        return self.__mul_pt(self, multiplier)
    
    __rmul__ = __mul__
    
    def __str__(self):
        return "P({}, {})".format(self.x, self.y)
    
    def __eq__(self, value):
        try:
            if value == None:
                if self.x == None and self.y == None:
                    return True
                elif self.x == None or self.y == None:
                    raise Exception("ERROR: Point object cannot have coordinate of None type!")
                return False
            else:
                if self.x == value.x and self.y == value.y:
                    return True
                return False
        except:
            return False

class Curve:
    """
    Class defining an elliptic curve with generator G point (if set)
    """    
    def __init__(self, a, b, p = None) -> None:
        self.a = a
        self.b = b
        self.p = p
        pass
    
    def __point_on_curve(self, x, y):
        result = (x**3 + self.a*x + self.b - y**2) % self.p
        if result == 0:
            return True
        else:
            return False
        
    def on_curve(self, x, y):
        return self.__point_on_curve(x, y)
    
    def list_points_over_finite_field(self) -> list:
        try:
            if self.p == None:
                raise Exception("You need to specify parameter p!")
            
            points = []
            
            for x in range(self.p):
                for y in range(self.p):
                    result = self.__point_on_curve(x, y)
                    if result == True:
                        points.append((x, y))
            return points
        except:
            self.p = input("Set value p: ")
            self.p = int(self.p)
            self.list_points_over_finite_field
            
    def set_generator(self, x, y):
        self.G = Point(x, y, self)
        print("Generator set properly to point G({}, {})".format(x, y))
        
# ------- MAIN --------

curr = registry.get_curve('secp192r1')        

curve = Curve(curr.a, curr.b, curr.field.p)
curve.set_generator(curr.g.x, curr.g.y)
print("-- CYCLIC GROUP --")
for it in range(10):
    print(it, curve.G*it)

Generator set properly to point G(602046282375688656758213480587526111916698976636884684818, 174050332293622031404857552280219410364023488927386650641)
-- CYCLIC GROUP --
0 P(None, None)
1 P(602046282375688656758213480587526111916698976636884684818, 174050332293622031404857552280219410364023488927386650641)
2 P(5369744403678710563432458361254544170966096384586764429448, 5429234379789071039750654906915254128254326554272718558123)
3 P(2915109630280678890720206779706963455590627465886103135194, 2946626711558792003980654088990112021985937607003425539581)
4 P(1305994880430903997305943738697779408316929565234787837114, 3981863977451150342116987835776121688410789618551673306674)
5 P(410283251116784874018993562136566870110676706936762660240, 1206654674899825246688205669651974202006189255452737318561)
6 P(4008504146453526025173196900303594155799995627910231899946, 3263759301305176906990806636587838100022690095020155627760)
7 P(3473339081378406123852871299395262476289672479707038350589, 21527131